<a href="https://colab.research.google.com/github/rosanth/Recommended_Sys/blob/Sajeevan/Reco_Sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf

In [30]:
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import fitz

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Sample job requirements (Multiple examples for better training)
job_descriptions = [
    "Looking for a candidate with knowledge in Python, Java, and Power BI.",
    "Seeking a Data Analyst with Power BI and SQL experience.",
    "Need a software engineer with knowledge in Java and Python.",
    "Hiring a business analyst skilled in Excel and Power BI.",
    "Need Cloud based Knowledge"
]

# Corresponding labels (1 = Suitable, 0 = Not Suitable)
labels = [1, 1, 1, 1, 0]

# Preprocess job descriptions
job_descriptions = [preprocess_text(desc) for desc in job_descriptions]

# Tokenization
max_words = 1000  # Max unique words in vocab
max_length = 100  # Max sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(job_descriptions)

# Convert job descriptions to sequences
X_train = tokenizer.texts_to_sequences(job_descriptions)
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')

y_train = np.array(labels)

# Define the Model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=32, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
model.fit(X_train, y_train, epochs=100, batch_size=2, verbose=1, validation_split=0.2)

# Load and extract resume text from a PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

# Clean and preprocess resume text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w.,]', ' ', text)
    return text.strip()

# Function to evaluate a resume
def evaluate_resume(resume_text):
    resume_text = preprocess_text(resume_text)
    input_data = tokenizer.texts_to_sequences([resume_text])
    input_data = pad_sequences(input_data, maxlen=max_length, padding='post')

    prediction = model.predict(input_data)[0][0]

    return "Resume is suitable for the job." if prediction > 0.7 else "Resume is not suitable for the job."

# Example usage: Extract, clean, and evaluate resume
pdf_path = "Sajeevan.pdf"  # Change this to your PDF file name
resume_text = extract_text_from_pdf(pdf_path)
resume_text = clean_text(resume_text)

# Print evaluation result
print(evaluate_resume(resume_text))
